In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'tqdm'

In [2]:
# Headers for all requests
headers = {
                'User-Agent':
                ('Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_3)'
                 ' AppleWebKit/537.36 (KHTML, like Gecko) '
                 'Chrome/48.0.2564.116 Safari/537.36')
}

#### Map options to query dict

Map for einfamilienhäusern

In [3]:
# base url for einfamilienhäusern
einfamilienhäusern = 'https://www.web.statistik.zh.ch/cms_handaenderungen/index.php?app=efh'

r = requests.get(einfamilienhäusern, headers=headers, verify=False)

# getting the 
einfamilienhäusern_soup =  BeautifulSoup(r.content, 'html.parser')
all_maps = einfamilienhäusern_soup.find_all('div', attrs = {'class':'auswahl'})

einfamilienhäusern_dict = {}

# final_dict for einfamilienhäusern
for a_map in all_maps:
    name = a_map.find('select')['name']
    einfamilienhäusern_dict[name] = {}
    for option in a_map.find_all('option'):
        if(option['value'] != '-1'):
            einfamilienhäusern_dict[name][option.text] = option['value']

Map for eigentumswohnungen

In [4]:
# base url for Eigentumswohnungen
eigentumswohnungen = 'https://www.web.statistik.zh.ch/cms_handaenderungen/index.php?app=efh'

r = requests.get(eigentumswohnungen, headers=headers, verify=False)

# getting the 
eigentumswohnungen_soup =  BeautifulSoup(r.content, 'html.parser')
all_maps = eigentumswohnungen_soup.find_all('div', attrs = {'class':'auswahl'})

eigentumswohnungen_dict = {}

# final_dict for eigentumswohnungen
for a_map in all_maps:
    name = a_map.find('select')['name']
    eigentumswohnungen_dict[name] = {}
    for option in a_map.find_all('option'):
        if(option['value'] != '-1'):
            eigentumswohnungen_dict[name][option.text] = option['value']

In [5]:
# We didn't need both, what a surprise, we get rid of one in the next cell
eigentumswohnungen_dict == einfamilienhäusern_dict

True

In [6]:
# the important dictionary
query_dict = eigentumswohnungen_dict

In [7]:
dict_list = list(query_dict)

In [8]:
dict_list

['GEBIET',
 'REGION',
 'HANDLRAUM',
 'VERKJAHR',
 'ZIMMERK',
 'BAUALTERK',
 'FLAEK',
 'ZONEK',
 'PARZK',
 'LAGE']

In [9]:
# parameters
params = {'GEBIET': 0,
'REGION': 0,
'HANDLRAUM': 0,
'app': 'app%3Defh',
'VERKJAHR': 0,
'ZIMMERK': 0,
'BAUALTERK': 0,
'FLAEK': 0,
'ZONEK': 0,
'PARZK': 0,
'LAGE': 0}

base_url = 'https://www.web.statistik.zh.ch/cms_handaenderungen/includes/ajax_all_results.php'

In [10]:
final_list = []

In [11]:
dict1 = query_dict[dict_list[0]]
dict2 = query_dict[dict_list[1]]
dict3 = query_dict[dict_list[2]]
dict4 = query_dict[dict_list[3]]
dict5 = query_dict[dict_list[4]]
dict6 = query_dict[dict_list[5]]
dict7 = query_dict[dict_list[6]]
dict8 = query_dict[dict_list[7]]
dict9 = query_dict[dict_list[8]]
dict10 = query_dict[dict_list[9]]

In [12]:
s = requests.Session()
for GEBIET in tqdm(query_dict[dict_list[0]].keys()):    
    for VERKJAHR in ['Alle Jahre', 2010, 2011, 2012, 2013, 2014]: 
                                                                                
        params['GEBIET'] = dict1[GEBIET]
        params['VERKJAHR'] = dict4[str(VERKJAHR)]
                                        
        r = s.post(base_url, data=params, verify=False)
        result_content = BeautifulSoup(r.text, 'html.parser')
        results = result_content.find_all('li')
        length = len(results)

        temp = {}
        temp['GEBIET'] = str(GEBIET)
        temp['VERKJAHR'] = str(VERKJAHR)
                                        
        if(length != 0):
            temp['YEAR'] = results[0].find('b').text
            temp['TOTAL'] = results[1].find('b').text
            temp['25 Quantile'] = results[2].find('b').text
            temp['Median'] = results[3].find('b').text
            temp['75 Quantile'] = results[4].find('b').text
        else:
            temp['YEAR'] = 0
            temp['TOTAL'] = 0
            temp['25 Quantile'] = 0
            temp['Median'] = 0
            temp['75 Quantile'] = 0

        final_list.append(temp.copy())


100%|██████████| 167/167 [04:47<00:00,  1.72s/it]


In [13]:
gemeinde_df = pd.DataFrame(final_list)

In [14]:
gemeinde_df

,25 Quantile,75 Quantile,GEBIET,Median,TOTAL,VERKJAHR,YEAR
0,720'000 sFr.,1'300'000 sFr.,Ganzer Kanton,945'000 sFr.,26'028,Alle Jahre,2007 - 2017
1,685'000 sFr.,1'200'000 sFr.,Ganzer Kanton,870'000 sFr.,2'581,2010,2010
2,698'750 sFr.,1'258'500 sFr.,Ganzer Kanton,900'000 sFr.,2'492,2011,2011
3,750'000 sFr.,1'350'000 sFr.,Ganzer Kanton,980'000 sFr.,2'365,2012,2012
4,770'000 sFr.,1'390'000 sFr.,Ganzer Kanton,994'570 sFr.,2'180,2013,2013
5,795'000 sFr.,1'380'000 sFr.,Ganzer Kanton,1'030'000 sFr.,2'299,2014,2014
6,629'000 sFr.,868'250 sFr.,Adlikon,772'000 sFr.,50,Alle Jahre,2007 - 2017
7,0,0,Adlikon,0,0,2010,0
8,0,0,Adlikon,0,0,2011,0
9,0,0,Adlikon,0,0,2012,0


In [1]:
r = requests.get('https://www.web.statistik.zh.ch/cms_handaenderungen/index.php?app=efh',
                  params = params, verify=False)

NameError: name 'requests' is not defined

testing

In [57]:
rslt = 'https://www.web.statistik.zh.ch/cms_handaenderungen/includes/ajax_all_results.php'

In [58]:
test = requests.get(rslt, params=params, verify=False)

/home/kirtan/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [59]:
s = BeautifulSoup(test.content, 'html.parser')

In [60]:
s

 ﻿<ul id="resultat_zeile"><li><span id="jahre" title="Jahr|Es werden nur Verkäufe der Jahre 2007 - 2017 angezeigt">Jahr:<b><br/> 2007 - 2017</b></span></li><li><span id="total" title="Verkäufe|Anzahl der Verkäufe mit dem gewählten Filter.">Verkäufe:<br/><b>50</b></span></li><li><span title="25% Quantil|25% der Verkäufe sind kleiner als 629'000 sFr.">25% Quantil: <br/><b>629'000 sFr.</b><br/> </span></li><li><span id="median" title="Median|50% der Preise der Verkäufe sind grösser, 50% kleiner als 772'000 sFr.">Median: <br/><b><span class="median">772'000 sFr.</span></b></span></li><li><span title="75% Quantil|75% der Verkäufe sind kleiner als 868'250 sFr.">75% Quantil: <br/><b>868'250 sFr.</b><br/></span></li></ul><p style="clear:both;"></p>

In [13]:
s.find_all('li')

NameError: name 's' is not defined

Define base parameter to start running queries

In [22]:
s = BeautifulSoup(r.content, 'html.parser')


In [23]:
s.find_all('ul', attrs = { 'id':'resultat_zeile'} )

[]

In [46]:
a